## Einleitung: Klassifikation von Fonds in Peergroups mit ML

### Problemstellung

Die Einordnung neuer Fonds in Peergroups erfolgt aktuell manuell durch Mitarbeiter. Dafür müssen Informationen aus dem Internet recherchiert und die Fonds individuell analysiert werden. Des Weiteren muss regelmäßig durch aufwendige Analysen geprüft werden, ob Fonds neu klassifiziert werden müssen. Mit steigender Zahl an Fonds und Peergroups wird dieser Prozess zunehmend aufwendig und fehleranfällig. Zusätzlich wird es bei zunehmender Anzahl an Peergroups schwierig zu erkennen, ob diese die Fonds noch klar genug trennen können.

### Ziel

Zielsetzung dieses Projektteils ist es, mithilfe von Machine-Learning-Modellen automatisch Vorschläge für die Zuordnung eines Fonds zu einer oder mehreren Peergroups zu erzeugen. Damit soll entweder der manuelle Aufwand entfallen oder der Mitarbeiter bei der Einordnung unterstützt werden.

Des Weiteren soll ein Clustering helfen zu prüfen, welche Fonds ein ähnliches Rendite-Risiko-Profil aufweisen. Dadurch kann erkannt werden, wie stark sich Fonds unterschiedlicher Peergroups in diesem Aspekt unterscheiden, und es können Rückschlüsse auf die Peergroup-Zusammensetzung gezogen werden.

### Feature-Auswahl

Für die Klassifikation wurden nur Daten genutzt, die für alle Fonds in der Breite verfügbar sind:
- rollierende Basis-Kennzahlen, die aus den Renditezeitreihen berechnet werden,
- Korrelationen zu unterschiedlichen Anlageklassen,
- sowie der Fondsname.

Detailliertere Informationen wie Strategietexte, Ratings und weitere Stammdaten wurden bewusst nicht genutzt, da diese nicht für alle Fonds verfügbar sind. Die Klassifizierung ist zudem vor allem bei internen Analysen über Produkte interessant, die nicht frei gehandelt werden und bei denen oft nur eine Rendite-Zeitreihe und der Fondsname zur Verfügung stehen.

### Vorgehen

Der Workflow gliedert sich in folgende Schritte:
1. Laden und Filtern der Daten: Fokus auf Peergroups mit ausreichend Datenbasis (mind. 50 Fonds).
2. Erstellung eines Feature-Sets aus verschiedenen Kennzahlen und Ähnlichkeitswerten.
3. Training von zwei Modelltypen (Random Forest und MLPClassifier) mit mehreren Feature-Kombinationen.
4. K-Means Clustering zur Gruppierung nach quantitativen Merkmalen.


## Importieren der nötigen Packete und Kennzahlen aus Kennzahlberechnung.py 

In [ ]:
# Pfade zu den berechneten Kennzahlen
import numpy as np
import pandas as pd
import re
import pickle
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans

## Daten laden und filtern (Vorbereitung für Machine Learning)

In diesem Abschnitt werden die berechneten Kennzahlen aus den benötigten Daten aus pickle-Dateien geladen und Fondsdaten aus der `Returns.xlsx`. Die Kennzahlen werden für die Features der ML-Modelle benötigt und die Fondsdaten, um die Fonds zu Filtern die für das Training der Modelle genutzt werden.

Da einige Peergroups zu wenige Fonds enthalten, um aussagekräftige Trainingsdaten zu liefern, werden nur Peergroups mit mehr als 50 Fonds berücksichtigt. Um die Fonds die verwendet werden sollen zu Filtern, wird `valid_ISIN` mit den Fonds definiert, die in einer Peergroup mit mehr als 50 Fonds sind.


In [22]:
path = "./Ergebnisse"

# Fondsdaten laden für Mapping der ISIN und Peergroup für die spätere Label-Zuordnung
returns_df = pd.read_excel('./Daten/Returns.xlsx')
returns_df.drop(index=[0, 1, 2, 3, 4], inplace=True)
returns_df.drop(['Ranking'], axis=1, inplace=True)
fonds_zu_peergroup = dict(zip(returns_df['ISIN'], returns_df['Peergroup']))

# Laden der berechneten Kennzahlen
with open('./Ergebnisse/Rendite.pkl', 'rb') as f:
    rendite_data = pickle.load(f)
with open('./Ergebnisse/Vola.pkl', 'rb') as f:
    vola_data = pickle.load(f)
with open('./Ergebnisse/Sharpe.pkl', 'rb') as f:
    sharpe_data = pickle.load(f)
with open('./Ergebnisse/Omega.pkl', 'rb') as f:
    omega_data = pickle.load(f)
with open('./Ergebnisse/Korrelationen.pkl', 'rb') as f:
    korrelationen_data = pickle.load(f)
with open('./Ergebnisse/WorstMonth.pkl', 'rb') as f:
    worstmonth_data = pickle.load(f)
with open('./Ergebnisse/MaxDD.pkl', 'rb') as f:
    maxdd_data = pickle.load(f)

# Filtern der Peergroups mit mehr als 50 Fonds, da nur diese für die Modelle genutzt werden
peergroup_counts = returns_df.groupby('Peergroup')['ISIN'].nunique()
peergroups_to_keep = peergroup_counts[peergroup_counts > 50].index

# Filtern der Fonds, die in den Peergroups sind, die für die Modelle genutzt werden
Renditen_filtered = returns_df[returns_df['Peergroup'].isin(peergroups_to_keep)].copy()
isin_zu_peergroup = dict(zip(Renditen_filtered['ISIN'], Renditen_filtered['Peergroup']))
valid_ISIN = set(Renditen_filtered['ISIN'].unique())

## Feature-Erstellung für das Modelltraining

In diesem Abschnitt werden die Features für die Machine-Learning-Modelle erstellt.

Zunächst werden die rollierenden Basiskennzahlen für die Zeitfenster 12, 24, 36 und 60 Monate extrahiert. Für jedes Zeitfenster wird dabei der jeweils letzte verfügbare Wert verwendet. Hierbei werden die kennzahlen die mit den Peergroup_Benchmarks nicht berücksichtigt, da diese durch die zurodnung zur Peergroup_Benchmarks eine Verbindung zur Peergroup die vorhergesagt werden soll haben. Die Basiskennzahlen sollen dem Model informationen über das Rendte-Risiko-Profil der Fonds liefern.

Zusätzlich werden vier nicht überlappende 12-Monats-Zeiträume vor dem letzten Zeitpunkt erstellt. Diese sollen helfen, Entwicklungen in der Vergangenheit als separate Zeitblöcke ins Modell einzubringen.

Außerdem werden die Korrelationen der Fonds zu allgemeinen Marktbenchmarks als weitere Features hinzugefügt. Diese dienen dazu informationen über das verhalten der Fonds gegenüber unterschiedlichen Anlageklassen zu liefern.

Zur Ergänzung der quantitativen Features wird ein Ähnlichkeitswert zwischen dem Fondsnamen und den Namen der Peergroups berechnet. Dafür wird eine TF-IDF-Vektorisierung auf den Namen durchgeführt und anschließend die Kosinus-Ähnlichkeit bestimmt. Dieser Schritt wird durchgeführt, da der Fondsname häufig Informationen über die Anlagestrategie des Fonds enthält.

Des Weiteren werden nur Fonds beibehalten, die mindestens 60 Monate Historie aufweisen, damit für jeden Fonds zu jedem Feature ein Wert vorhanden ist.


In [23]:
# Zeiträume für die verwendeten Kennzahlen definieren
windows = ['12M', '24M', '36M', '60M']

# feature DataFrame für die Sammlung aller Features initalisieren
features = pd.DataFrame()

for window in windows:
    # Extrahieren der Basiskennzahlen über die Zeiträume für den jeweils aktuellsten Monat der Zeitreihe
    features[f'Rendite_{window}'] = rendite_data[window].iloc[-1]
    features[f'Vola_{window}'] = vola_data[window].iloc[-1]
    features[f'Sharpe_{window}'] = sharpe_data[window].iloc[-1]
    features[f'Omega_{window}'] = omega_data[window].iloc[-1]
    features[f'WorstMonth_{window}'] = worstmonth_data[window].iloc[-1]
    features[f'MaximumDrawdown_{window}'] = maxdd_data[window].iloc[-1]
    # Extrahieren der Korrelationen mit den allgemeinen Benchmarks
    for index in korrelationen_data["Korr_12M"].index:
        features[f'Korr_{index}_{window}'] = korrelationen_data[f'Korr_{window}'].loc[index]

# Extrahieren der vier nicht überlappenden 12 Monats zeiträume vor dem aktuellen Zeitraum
for i in range(4):
    suffix = f"12M_Block{i+1}"
    features[f'Rendite_{suffix}'] = rendite_data['12M'].iloc[-(12 * (i + 1))]
    features[f'Sharpe_{suffix}'] = sharpe_data['12M'].iloc[-(12 * (i + 1))]
    features[f'Vola_{suffix}'] = vola_data['12M'].iloc[-(12 * (i + 1))]

# Mapping der FondsID zur ISIN zum Fondsnamen
fondsid_zu_isin = returns_df[['FondsID', 'ISIN']].drop_duplicates().set_index('FondsID')['ISIN'].to_dict()
isin_zu_name = returns_df[['ISIN', 'Fondsname']].drop_duplicates().set_index('ISIN')['Fondsname']
fonds_namen = features.index.to_series().map(fondsid_zu_isin).map(isin_zu_name).fillna('').astype(str)
peergroup_texts = list(peergroups_to_keep)

# TF-IDF Vektorisierung
vectorizer = TfidfVectorizer()
tfidf_fonds = vectorizer.fit_transform(fonds_namen.values)
tfidf_peergroups = vectorizer.transform(peergroup_texts)

# Berechnung der Kosinus-Ähnlichkeit
similarities = cosine_similarity(tfidf_fonds, tfidf_peergroups)
similarity_df = pd.DataFrame(similarities, index=fonds_namen.index, columns=[f'name_sim_{pg}' for pg in peergroup_texts])

# Mit Features zusammenführen
features = features.join(similarity_df)

# Mapping der FondsID zur ISIN
fondsid_zu_isin = returns_df.reset_index().set_index('FondsID')['ISIN'].to_dict()
features.index = features.index.map(fondsid_zu_isin)

# droppen von Fonds, die weniger als 60 Monate Historie aufweisen, damit alle Fonds Werte für für alle Features aufweisen
features.dropna(subset=["Rendite_60M"], inplace=True)
features = features.loc[features.index.isin(valid_ISIN)]


## Modelltraining: Peergroup-Klassifikation mit Random Forest & MLP

In diesem Abschnitt werden Machine-Learning-Modelle trainiert, um die Fonds in Peergroups einzuordnen. Dabei handelt es sich um eine Multi-Label-Klassifikationsaufgabe, da ein Fonds mehreren Peergroups zugeordnet sein kann.

Die Zielvariable (`y_multi`) wird mithilfe von One-Hot-Encoding auf Basis der ISIN-Peergroup-Zuordnung erstellt.

Zur Untersuchung der Modellleistung werden verschiedene Feature-Kombinationen getestet:
- Basis: Nur die aktuellen Kennzahlen über verschiedene Zeiträume
- Basis+Rolling12M: Zusätzlich vier nicht überlappende 12M-Zeitblöcke
- Basis+NameSim: Zusätzlich die Ähnlichkeitswerte basierend auf dem Fondsnamen
- Basis+Rolling12M+NameSim: Kombination aller verfügbaren Features

Zwei Modelltypen werden eingesetzt, um verschiedene Ansätze zu vergleichen:
- Ein Random Forest Classifier (performant, robust)
- Ein MLPClassifier (neuronales Netz), um auch komplexere Muster erkennen zu können


Für beide Modelle werden mit `RandomizedSearchCV` passende Hyperparameter-Kombinationen getestet. Anschließend werden Accuracy und F1-Scores (macro & samples) für die Modelle auf dem Testdatensatz ausgegeben.



In [ ]:
# erstellen der Peergroup-Labels (Zielvariablen) im Multi-Label Format mit One-Hot-Encoding
peergroup_series = pd.Series(fonds_zu_peergroup).loc[features.index]
y_multi = pd.get_dummies(peergroup_series).astype(int).sort_index()

# Initialisierung des Scalers zur Standardisierung der Features
scaler = StandardScaler()

# Definitieren der verschiedenen Feature-Kombinationen
feature_sets = {
    'Basis': features.filter(regex='^(Rendite|Vola|Sharpe|Omega|WorstMonth|MaximumDrawdown)_'),
    'Basis+Rolling12M': features.filter(regex='^(Rendite|Vola|Sharpe|Omega|WorstMonth|MaximumDrawdown|12M_Block)_'),
    'Basis+NameSim': features.filter(regex='^(Rendite|Vola|Sharpe|Omega|WorstMonth|MaximumDrawdown|name_sim_)'),
    'Basis+Rolling12M+NameSim': features
}
# Hyperparameter für Random Forest
rf_params = {
    'estimator__n_estimators': [100, 200],
    'estimator__max_depth': [30, 50],
    'estimator__min_samples_split': [2, 5],
}

# Hyperparameter für MLP (Neural Network)
mlp_params = {
    'estimator__hidden_layer_sizes': [(512, 256),(256, 128), (128, 64)],
    'estimator__activation': ['relu', 'tanh'],
    'estimator__alpha': [0.0001, 0.001],
    'estimator__learning_rate_init': [0.001, 0.0001],
}

# Multi-Output-Wrapper für die Modelle (mehrere Peergroup-Labels gleichzeitig)
multi_rf = MultiOutputClassifier(RandomForestClassifier(n_jobs=-1, random_state=42))
multi_mlp = MultiOutputClassifier(MLPClassifier(max_iter=1000, early_stopping=True, random_state=42))

# Dictionaries zur Speicherung der Ergebnisse initialisieren
results_rf = {}
results_mlp = {}

# Trainings-Loop über alle definierten Feature-Sets
for name, X_features in feature_sets.items():
    print(f"\n Testing Feature-Set: {name}")

    # Features standardisieren
    X_scaled = pd.DataFrame(scaler.fit_transform(X_features), index=X_features.index, columns=X_features.columns)
    
    # Aufteilung in Trainings- und Testdaten
    train_ids, test_ids = train_test_split(X_scaled.index, test_size=0.2, random_state=42)
    X_train, X_test = X_scaled.loc[train_ids], X_scaled.loc[test_ids]
    y_train_split, y_test_split = y_multi.loc[train_ids], y_multi.loc[test_ids]

    # Trainieren des Random Forest Modell mit RandomizedSearchCV
    rf_model = RandomizedSearchCV(multi_rf, rf_params, cv=3, n_jobs=-1, n_iter=6, random_state=42)
    rf_model.fit(X_train, y_train_split)
    preds_rf = rf_model.predict(X_test)
    
    # Evaluation des RF-Modells
    acc_rf = accuracy_score(y_test_split, preds_rf)
    f1_macro_rf = f1_score(y_test_split, preds_rf, average='macro')
    f1_samples_rf = f1_score(y_test_split, preds_rf, average='samples')
    results_rf[name] = {'Accuracy': acc_rf, 'F1_macro': f1_macro_rf, 'F1_samples': f1_samples_rf}

    # Trainieren des MLP-Modell (Neural Network) mit RandomizedSearchCV
    mlp_model = RandomizedSearchCV(multi_mlp, mlp_params, cv=3, n_jobs=-1, n_iter=6, random_state=42)
    mlp_model.fit(X_train, y_train_split)
    preds_mlp = mlp_model.predict(X_test)
    
    # Evaluation des MLP-Modells
    acc_mlp = accuracy_score(y_test_split, preds_mlp)
    f1_macro_mlp = f1_score(y_test_split, preds_mlp, average='macro')
    f1_samples_mlp = f1_score(y_test_split, preds_mlp, average='samples')
    results_mlp[name] = {'Accuracy': acc_mlp, 'F1_macro': f1_macro_mlp, 'F1_samples': f1_samples_mlp}

    print(f"→ RF: Acc: {acc_rf:.4f} | F1_macro: {f1_macro_rf:.4f} | F1_samples: {f1_samples_rf:.4f}")
    print(f"→ MLP: Acc: {acc_mlp:.4f} | F1_macro: {f1_macro_mlp:.4f} | F1_samples: {f1_samples_mlp:.4f}")

print("\n Vergleich Random Forest:")
print(pd.DataFrame(results_rf).T.sort_values("F1_samples", ascending=False))

print("\n Vergleich MLPClassifier:")
print(pd.DataFrame(results_mlp).T.sort_values("F1_samples", ascending=False))



 Testing Feature-Set: Basis


/Users/johntitz/Desktop/Datenanalyse/venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


→ RF: Acc: 0.5582 | F1_macro: 0.5951 | F1_samples: 0.5586
→ MLP: Acc: 0.4965 | F1_macro: 0.4712 | F1_samples: 0.5156

 Testing Feature-Set: Basis+Rolling12M


/Users/johntitz/Desktop/Datenanalyse/venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/johntitz/Desktop/Datenanalyse/venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


→ RF: Acc: 0.5582 | F1_macro: 0.5951 | F1_samples: 0.5586
→ MLP: Acc: 0.4965 | F1_macro: 0.4712 | F1_samples: 0.5156

 Testing Feature-Set: Basis+NameSim


/Users/johntitz/Desktop/Datenanalyse/venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/johntitz/Desktop/Datenanalyse/venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


→ RF: Acc: 0.5644 | F1_macro: 0.6010 | F1_samples: 0.5650
→ MLP: Acc: 0.5035 | F1_macro: 0.4631 | F1_samples: 0.5241

 Testing Feature-Set: Basis+Rolling12M+NameSim


/Users/johntitz/Desktop/Datenanalyse/venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/johntitz/Desktop/Datenanalyse/venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


→ RF: Acc: 0.5869 | F1_macro: 0.6222 | F1_samples: 0.5872
→ MLP: Acc: 0.5574 | F1_macro: 0.5455 | F1_samples: 0.5829

 Vergleich Random Forest:
                          Accuracy  F1_macro  F1_samples
Basis+Rolling12M+NameSim  0.586865  0.622154    0.587239
Basis+NameSim             0.564412  0.601017    0.564973
Basis                     0.558237  0.595127    0.558612
Basis+Rolling12M          0.558237  0.595127    0.558612

 Vergleich MLPClassifier:
                          Accuracy  F1_macro  F1_samples
Basis+Rolling12M+NameSim  0.557395  0.545534    0.582889
Basis+NameSim             0.503508  0.463092    0.524090
Basis                     0.496492  0.471216    0.515577
Basis+Rolling12M          0.496492  0.471216    0.515577


/Users/johntitz/Desktop/Datenanalyse/venv/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## K-Means Clustering zur Analyse von Fondsgruppen

In diesem Abschnitt wird ein K-Means-Clustering durchgeführt, um zu prüfen, welche Fonds ein ähnliches Rendite-Risiko-Profil aufweisen. Dafür werden die aktuellsten Werte der Kennzahlen Rendite, Volatilität und Sharpe-Ratio verwendet. Die Cluster können genutzt werden um die Zusammensetzung der Benchmarks zu prüfen.

Nach dem Clustering wird für jedes Cluster analysiert:
- Wie viele Fonds diesem Cluster zugeordnet wurden
- Welche Peergroups innerhalb des Clusters am häufigsten vorkommen

In [ ]:

# Auswahl der Features für das Clustering
clustering_features = [
    col for col in features.columns
    if any(key in col for key in ['Rendite', 'Vola', 'Sharpe']) and 'Block' not in col
]

# Standardisierung der Clustering-Daten
X_cluster = StandardScaler().fit_transform(features[clustering_features])

# Clustering mit K-Means
n_clusters = 20
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
features['Cluster'] = kmeans.fit_predict(X_cluster)

# Cluster-Analyse: Zuordnung von Fonds zu Cluster & Peergroup
labels = pd.Series(fonds_zu_peergroup).loc[features.index]
cluster_analysis = pd.DataFrame({
    'FondsID': features.index,
    'Cluster': features['Cluster'],
    'Peergroup': labels.values
})

# Ausgabe: Häufigkeit pro Cluster & Peergroup-Verteilung
for i in range(n_clusters):
    cluster_data = cluster_analysis[cluster_analysis['Cluster'] == i]
    print(f"\nCluster {i} Analysis:")
    print(f"Number of funds: {len(cluster_data)}")
    print("\nTop Peergroups:")
    print(cluster_data['Peergroup'].value_counts().head())



Cluster 0 Analysis:
Number of funds: 869

Top Peergroups:
Peergroup
Equity USA                 223
Sustainability Global      146
Equity Global Developed    103
Sustainability USA          95
Equity Global               68
Name: count, dtype: int64

Cluster 1 Analysis:
Number of funds: 1331

Top Peergroups:
Peergroup
Sustainability Bonds Europe Corporate          128
Corporate Bonds Europe                         119
Diversified Bonds Global Hedged                 96
Multi-Asset Global Bond Bias                    86
Sustainability Multi-Asset Global Bond Bias     70
Name: count, dtype: int64

Cluster 2 Analysis:
Number of funds: 923

Top Peergroups:
Peergroup
Sustainability Europe    171
Equity Europe            157
Equity Switzerland       106
Equity Eurozone           53
Commodity Diversified     36
Name: count, dtype: int64

Cluster 3 Analysis:
Number of funds: 1142

Top Peergroups:
Peergroup
Multi-Asset Global Moderate Equity Bias                   167
Multi-Asset Global Bond Bia

## Fazit: Klassifikation von Fonds in Peergroups mit Machine Learning

Ziel dieses Projektteils war es, ein Modell zu entwickeln, das neue Fonds automatisch einer oder mehreren passenden Peergroups zuordnet. Hintergrund dafür war, den manuellen Aufwand bei der Klassifikation zu reduzieren. Dafür wurden nur Features verwendet, die für alle Fonds breit verfügbar sind.

Die Ergebnisse der getesteten Modelle (Random Forest und MLPClassifier) zeigten jedoch, dass diese insgesamt keine ausreichende Trennschärfe aufweisen, um in der Praxis eingesetzt werden zu können. Insbesondere der F1-Score, der bei keiner Variante über 0,63 lag, zeigt, dass die Modelle Schwierigkeiten haben, verlässlich zu klassifizieren. Die Unterschiede zwischen den Peergroups scheinen auf Basis der Rendite-Risiko-Kennzahlen allein nicht ausreichend stark zu sein, um sie zuverlässig zu trennen. Auch der Einsatz des Fondsnamens als textbasiertes Zusatzfeature konnte das Modell nur bedingt verbessern.

**Random Forest:**  
zeigte insgesamt bessere Ergebnisse als das neuronale Netz. Die besten Resultate wurden mit der Kombination aus Basiskennzahlen, Rolling-Blöcken und Namensähnlichkeit erzielt:

| Feature-Set                  | Accuracy | F1_macro | F1_samples |
|-----------------------------|----------|----------|------------|
| Basis+Rolling12M+NameSim    | 0.587    | 0.622    | 0.587      |
| Basis+NameSim               | 0.564    | 0.601    | 0.565      |
| Basis                       | 0.558    | 0.595    | 0.559      |
| Basis+Rolling12M            | 0.558    | 0.595    | 0.559      |

**MLPClassifier:**  
schnitt insgesamt schlechter ab:

| Feature-Set                  | Accuracy | F1_macro | F1_samples |
|-----------------------------|----------|----------|------------|
| Basis+Rolling12M+NameSim    | 0.557    | 0.546    | 0.583      |
| Basis+NameSim               | 0.504    | 0.463    | 0.524      |
| Basis                       | 0.496    | 0.471    | 0.516      |
| Basis+Rolling12M            | 0.496    | 0.471    | 0.516      |

Ein möglicher Grund für die Ergebnisse ist, dass die betrachteten Märkte zu stark korreliert sind, um die Fonds allein über die Kennzahlen aus den Renditezeitreihen unterscheiden zu können. Auch die Fondsnamen scheinen nicht ausreichend Informationen über die Anlagestrategie zu liefern. Zusätzlich könnten die Peergroups auch zu kleinteilig definiert sein, wodurch deutlich mehr Informationen nötig wären, um die Unterschiede zwischen ihnen sauber zu erfassen. Vor diesem Hintergrund können die Modelle nicht genutzt werden, um die manuelle Klassifikation zu ersetzen.

### Clustering zur Analyse von Peergroups

Das zusätzlich durchgeführte K-Means-Clustering zeigte, dass Fonds einer Peergroup häufig in unterschiedlichen Clustern landen. Gleichzeitig fanden sich Fonds aus verschiedenen Peergroups, aber mit ähnlicher Ausrichtung (z. B. gleiche Region oder Anlageklasse), im selben Cluster wieder. Auch das spricht dafür, dass sich Peergroups mit den hier genutzten quantitativen Informationen nicht klar abgrenzen lassen.

Die Ergebnisse bestätigen die Annahme, dass zusätzliche qualitative Informationen notwendig wären, um eine verlässliche Peergroup-Klassifikation per Machine Learning zu ermöglichen.
